In [1]:
import numpy as np
import torch
from itertools import product
from collections import defaultdict
from agents import *
from config import Config

In [2]:
from tqdm import tqdm

def get_next_v(v_bar = 1, sigma_v = 1):
    return v_bar + np.random.normal(scale = sigma_v)

def simulate(
        T = 1000000, config = None,     # simulation time and config file
        continue_simulation = False,    # continue simulation if True
        save_path = None                # path to save the simulation
        ):
    assert config is not None, "Config file required"
    I = config.I
    Np = config.Np
    Nv = config.Nv
    Nx = config.Nx
    sigma_u = config.sigma_u
    if type(continue_simulation) == str:
        log = np.load(continue_simulation, allow_pickle=True).item()
        
        v_hist = np.zeros(T)
        p_hist = np.zeros(T)
        z_hist = np.zeros(T)
        x_hist = np.zeros((I, T))
        # y_hist = np.zeros((I, T))
        u_hist = np.zeros(T)
        profit_hist = np.zeros((I, T))
        t0 = 0

        informed_agents = log["agents"]["informed"]
        noise_agent = log["agents"]["noise"]
        preferred_habitat_agent = log["agents"]["preferred_habitat"]
        market_maker = log["agents"]["market_maker"]

        _state = log["last_state"]

        convergence_counter = log['convergence_counter']
    # elif type(continue_simulation) == dict:
    #     log = continue_simulation
    #     v_hist = log["v"]
    #     p_hist = log["p"]
    #     z_hist = log["z"]
    #     x_hist = log["x"]
    #     y_hist = log["y"]
    #     profit_hist = log["profit"]
    #     t0 = len(v_hist)
    #     informed_agents = log["agents"]["informed"]
    #     noise_agent = log["agents"]["noise"]
    #     preferred_habitat_agent = log["agents"]["preferred_habitat"]
    #     market_maker = log["agents"]["market_maker"]
    #     _state = log["last_state"]

    #     profit_hist = np.concatenate((profit_hist, np.zeros((I, T))), axis=1)
    #     v_hist = np.concatenate((v_hist, np.zeros(T)))
    #     p_hist = np.concatenate((p_hist, np.zeros(T)))
    #     z_hist = np.concatenate((z_hist, np.zeros(T)))
    #     x_hist = np.concatenate((x_hist, np.zeros((I, T))), axis=1)
    #     y_hist = np.concatenate((y_hist, np.zeros((I, T))), axis=1)

    #     convergence_counter = log['convergence_counter']
        
    elif continue_simulation == False:

        market_maker = AdaptiveMarketMaker(config)
        noise_agent = NoiseAgent(config)
        preferred_habitat_agent = PreferredHabitatAgent(config)
        informed_agents = [InformedAgent(config) for _ in range(I)]
        _state = (np.random.choice(Np), np.random.choice(Nv))

        # log histories
        v_hist = np.zeros(T)
        p_hist = np.zeros(T)
        z_hist = np.zeros(T)
        x_hist = np.zeros((I, T))
        # y_hist = np.zeros((I, T))
        u_hist = np.zeros(T)
        profit_hist = np.zeros((I, T))
        t0 = 0
        convergence_counter = 0
    else:
        raise ValueError("Invalid value for continue_simulation")
    for agent in informed_agents:
        agent.convergence_counter = convergence_counter
    if save_path is None:
        save_path = '/Users/moyuanchen/Documents/thesis/data.npy'

    for t in tqdm(range(T), desc="Simulation Progress"):
        yt = []
        _p, _v = informed_agents[0].p_discrete[_state[0]], informed_agents[0].v_discrete[_state[1]]
        v_hist[t+t0] = _v
        p_hist[t+t0] = _p
        _x = []
        for idx, agent in enumerate(informed_agents):
            x = agent.get_action(_state)
            xd = agent.x_discrete[x]
            yt.append(xd)
            _x.append(x)

            x_hist[idx, t + t0] = xd
            # y_hist[idx, t + t0] = yt[-1]
        ut = noise_agent.get_action()
        u_hist[t+t0] = ut
        yt_sum = np.sum(yt) + ut
        # print(yt_sum)
        zt = preferred_habitat_agent.get_action(_p)

        z_hist[t+t0] = zt

        market_maker.update(_v, _p, zt, yt_sum)
        pt = market_maker.determine_price(yt_sum)
        vt = get_next_v()
        next_state = informed_agents[0].continuous_to_discrete(pt, vt)
        for idx, agent in enumerate(informed_agents):
            reward = (_v - pt) * yt[idx]
            agent.update(_state, _x[idx], reward, next_state)
            profit_hist[idx, t + t0] = reward

        _state = next_state
    convergence = min([agent.convergence_counter for agent in informed_agents])
    log = {
        "v": v_hist,
        "p": p_hist,
        "z": z_hist,
        "x": x_hist,
        "u": u_hist,
        "profit": profit_hist,
        "last_state": _state,
        "convergence_counter": convergence
    }
    agents = {
        "informed": informed_agents,
        "noise": noise_agent,
        "preferred_habitat": preferred_habitat_agent,
        "market_maker": market_maker
    }
    log["agents"] = agents
    np.save(save_path, log)
    # print(max_c)
    return log, agents

#### Simulate $\sigma_u = 0.1$

In [ ]:
config = Config()
config.sigma_u = 0.1

_save_path = '/Users/moyuanchen/Documents/thesis/sim_1/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_1/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_1/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [02:34<00:00, 3236.91it/s]


Partition 1 completed, convergence counter: 36


Simulation Progress: 100%|██████████| 500000/500000 [03:12<00:00, 2592.48it/s]


Partition 2 completed, convergence counter: 44


Simulation Progress: 100%|██████████| 500000/500000 [03:09<00:00, 2643.77it/s]


...

Simulation Progress: 100%|██████████| 500000/500000 [02:25<00:00, 3430.49it/s]


Partition 22 completed, convergence counter: 48711


Simulation Progress: 100%|██████████| 500000/500000 [02:29<00:00, 3350.59it/s]


Partition 23 completed, convergence counter: 112749


Simulation Progress:  96%|█████████▌| 480712/500000 [02:21<00:06, 3205.20it/s]

In [5]:
config = Config()
config.sigma_u = 100

_save_path = '/Users/moyuanchen/Documents/thesis/sim_2/data_part_0.npy'
log, agents = simulate(config = config, T = 500000, save_path=_save_path)
convergence_threshold = 1000000
convergence = 0
partitions = 1
while convergence < convergence_threshold:
    save_path = f'/Users/moyuanchen/Documents/thesis/sim_2/data_part_{partitions}.npy'
    log, agents = simulate(config = config, T=500000, continue_simulation=_save_path, save_path=save_path)
    _save_path = save_path
    
    convergence = log['convergence_counter']
    print(f"Partition {partitions} completed, convergence counter: {convergence}")
    partitions += 1

# save config
np.save('/Users/moyuanchen/Documents/thesis/sim_1/config.npy', config)

Simulation Progress: 100%|██████████| 500000/500000 [06:17<00:00, 1325.23it/s]


Partition 1 completed, convergence counter: 42


Simulation Progress: 100%|██████████| 500000/500000 [03:49<00:00, 2179.40it/s]


Partition 2 completed, convergence counter: 7


Simulation Progress: 100%|██████████| 500000/500000 [07:29<00:00, 1111.88it/s]


...

Partition 74 completed, convergence counter: 584958


Simulation Progress: 100%|██████████| 500000/500000 [02:49<00:00, 2958.50it/s]


Partition 75 completed, convergence counter: 963041


Simulation Progress: 100%|██████████| 500000/500000 [02:58<00:00, 2793.57it/s]

Partition 76 completed, convergence counter: 1342911
